# 09_mle_estimation_v5

In [1]:
import numpy as np
import pandas as pd
import fs_qe as fq

# 1 theorem

## 1.1 log-likelihood function

The law of motion for the firm size is

$$
\ln \frac{S_{t+1}}{S_{t}} = a + b \frac{1}{\ln S_t} \tag{1}
$$
where
- $S_t$ is the firm size at time $t$
- $a \sim N(\mu_{a}, \sigma^2_{a})$
- $b \sim N(\mu_{b}, \sigma^2_{b})$

Let $X_t := \frac{1}{\ln S_t}$ and $Y_t:=\ln \frac{S_{t+1}}{S_{t}}$.
The above law of motion can be rewritten as
$$
Y_t = a + b X_t \tag{2}
$$
while other intepretations are the same.



For the sum of independent random variables (you can find the wikipedia [here](https://en.wikipedia.org/wiki/Sum_of_normally_distributed_random_variables)), we have the following conclusion that 

if
$$
X_i \sim_{iid} N(\mu_i, \sigma^2_i), i=1, \cdots, n
$$
then
$$
\sum^n_{i=1} a_i X_i \sim N( \sum^n_{i=1} a_i \mu_i, \sum^n_{i=1} (a_i \sigma_i)^2 )
$$

The above normal regression model can be regarded, equivalently, as a statement about the density of $Y$ given $X$.

The conditional density is
$$
f_{Y|X} (y |x, \mu_{a}, \mu_{b}, \sigma^2_{a}, \sigma^2_{b}) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp (- \frac{1}{2 \sigma^2} (y - \mu_{a} - \mu_{b} x )^2 ) \tag{3}
$$
where
- $\sigma^2 = \sigma^2_{a} + \sigma^2_{b} (x)^2$

The likelihood function is
$$
\mathscr L (\mu_{a}, \mu_{b}, \sigma^2_{a}, \sigma^2_{b} | x, y) = \prod^n_{i=1} \frac{1}{\sqrt{2 \pi \sigma^2}} \exp (- \frac{1}{2 \sigma^2} (y - \mu_{a} - \mu_{b} x )^2 )  \tag{4}
$$

Taking log of $(4)$, we have the log-likelihood function
$$
L (\mu_{a}, \mu_{b}, \sigma^2_{a}, \sigma^2_{b} | x, y) = \sum^n_{i=1}\ln \frac{1}{\sqrt{2 \pi \sigma^2}} \exp (- \frac{1}{2 \sigma^2} (y - \mu_{a} - \mu_{b} x )^2 )  \tag{5}
$$

## 1.2 tail index estimation by the [Kesten-Goldien Theorem](https://python.quantecon.org/kesten_processes.html#The-Kesten%E2%80%93Goldie-Theorem)

Recall equation [(1)](#), as $S_t \to \infty$, we have

$$
\ln \frac{S_{t+1}}{S_{t}} = a \tag{6}
$$

Taking logarithm of both yields

$$
\frac{S_{t+1}}{S_{t}} = \exp (a) \tag{7}
$$

By [Kesten-Goldien Theorem](https://python.quantecon.org/kesten_processes.html#The-Kesten%E2%80%93Goldie-Theorem), we have a positive constant $\alpha$, which is the tail index of the Pareto distribution followed by the stationary firm size distribution, such that

$$
\mathbb E (\exp (a)^{\alpha} ) = 1 \tag{8}
$$

By the property of exponential functions, $a \sim N(\mu_a, \sigma^2_a)$ and lognormal distribution, let $\epsilon \sim N(0, 1)$ equation [(8)](#) can be rewritten as

$$
\mathbb E (\exp (a)^{\alpha} ) =  \mathbb E (\exp (\mu_a + \sigma_a \epsilon)^{\alpha} )\\
= \mathbb E (\exp (\mu_a \alpha + \sigma_a \epsilon \alpha) ) \\
= \exp (  \alpha \mu_a + \frac{\sigma^2_a \alpha^2}{2} ) =1 \tag{9}
$$
or equivalently
$$
\exp (  \alpha \mu_a + \frac{\sigma^2_a \alpha^2}{2} ) =1 \tag{10}
$$

Taking logarithm of both sides of [(10)](#) yields

$$
\alpha \mu_a + \frac{\sigma^2_a \alpha^2}{2} =0 \tag{11}
$$

Rearrange [(11)](#), we get

$$
\alpha = - \frac{2 \mu}{\sigma^2} \tag{12}
$$

For the following parts, we estimate the tail index $\alpha$ of the stationary distribution using equation [(12)](#).

# 2 implementation
## 2.0 functions

## 2.1 implementation 1: without chopping

The file ``usa.csv`` comes from ``df7`` of [02_firm_size_plot_approximation_orbis_us](https://github.com/jstac/inequality_dynamics/blob/master/firm_size_empirics/02_firm_size_plot_approximation_orbis_us.ipynb) saved in our repo.

### 2.1.1 case i: mle with whole sample

In [2]:
df = pd.read_csv('data/usa.csv')
opt_res, μ_σ, se = fq.mle_mean_var_se(df, x='logtasset_inv', y='fgrow_log')

The following shows the maximum likelihood estimators for $(\mu_{a}, \mu_{b}, \sigma^2_{a}, \sigma^2_{b})$, respectively:

In [3]:
print(opt_res)
print('μ_a, μ_b, σ^2_a,  σ^2_b', μ_σ[0], μ_σ[1], μ_σ[2],  μ_σ[3])

      fun: 1022697.3896839675
 hess_inv: array([[ 9.39936266e-04,  5.94488585e-03, -2.30456423e-04,
        -8.08297212e-03],
       [ 5.94488585e-03,  8.68205532e-01, -3.94388916e-02,
        -6.39376679e-01],
       [-2.30456423e-04, -3.94388916e-02,  2.33016824e-03,
         2.65156663e-02],
       [-8.08297212e-03, -6.39376679e-01,  2.65156663e-02,
         5.38438236e-01]])
      jac: array([ 3.8203125, -0.3125   , -2.0703125,  0.09375  ])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 744
      nit: 47
     njev: 122
   status: 2
  success: False
        x: array([  -4.30609945,  -16.4868412 ,    6.98391849, 2724.20439276])
μ_a, μ_b, σ^2_a,  σ^2_b -4.306099452551644 -16.486841201415267 6.98391848920708 2724.204392764138


The following shows the standard errors for the estimators of $(\mu_{\alpha}, \mu_{\beta}, \sigma^2_{\alpha}, \sigma^2_{\beta})$, respectively:

In [4]:
print(se)

[0.03065838 0.93177547 0.04827182 0.73378351]


The following shows the estimated tail index $\alpha$ using the estimators $(\mu_a, \sigma^2_a)$ and equation [(12)]() above.

In [5]:
α = fq.alpha(μ_σ[0], μ_σ[1])
print('The tail index α of the stationary firm size distribution is', α)

The tail index α of the stationary firm size distribution is -0.5223680388432441


### 2.1.2 case ii: mle with most-recent observation for each firm in sample

Next, we keep the only most-recent observation for each firm to avoid time correlation.

In [6]:
df1 = df.groupby('id', as_index=False).tail(1)
opt_res1, μ_σ1, se1 = fq.mle_mean_var_se(df1, x='logtasset_inv', y='fgrow_log')

In [7]:
df1.to_csv('data/usa_select.csv')

The following shows the maximum likelihood estimators for $(\mu_{a}, \mu_{b}, \sigma^2_{a}, \sigma^2_{b})$, respectively, in this new case:

In [8]:
print(opt_res1)
print('μ_a, μ_b, σ^2_a,  σ^2_b', μ_σ1[0], μ_σ1[1], μ_σ1[2],  μ_σ1[3])

      fun: 137271.02156124418
 hess_inv: array([[ 2.25581736e-01, -4.11673566e+00,  1.68863789e-01,
        -1.49725292e+01],
       [-4.11673566e+00,  1.02384741e+02, -4.71013011e+00,
         3.68638434e+02],
       [ 1.68863789e-01, -4.71013011e+00,  2.44030921e-01,
        -1.69076149e+01],
       [-1.49725292e+01,  3.68638434e+02, -1.69076149e+01,
         1.32766584e+03]])
      jac: array([ 1.37695312,  0.1484375 , -2.07226562, -0.00976562])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 421
      nit: 46
     njev: 70
   status: 2
  success: False
        x: array([ -10.26413664,   53.40081162,    6.13113431, 3145.7609673 ])
μ_a, μ_b, σ^2_a,  σ^2_b -10.264136644088985 53.40081162035649 6.131134310476635 3145.760967302879


The following shows the standard errors for the estimators of $(\mu_{a}, \mu_{b}, \sigma^2_{a}, \sigma^2_{b})$, respectively:

In [9]:
print(se1)

[ 0.47495446 10.11853455  0.49399486 36.4371492 ]


The following shows the estimated tail index $\alpha$ using the estimators $(\mu_a, \sigma^2_a)$ and equation [(12)]() above.

In [10]:
α = fq.alpha(μ_σ1[0], μ_σ1[1])
print('The tail index α of the stationary firm size distribution is', α)

The tail index α of the stationary firm size distribution is 0.38441875067592707


## 2.2 implementation 2: with chopping

The file ``usa_chop_log8.csv`` comes from ``df8`` of [02_firm_size_plot_approximation_orbis_us](https://github.com/jstac/inequality_dynamics/blob/master/firm_size_empirics/02_firm_size_plot_approximation_orbis_us.ipynb) saved in our repo.

### 2.2.1 case i: mle with whole sample

In [11]:
%%time
df2 = pd.read_csv('data/usa_chop_log8.csv')
opt_res2, μ_σ2, se2 = fq.mle_mean_var_se(df2, x='logtasset_inv', y='fgrow_log')

CPU times: user 1min 4s, sys: 4.99 s, total: 1min 9s
Wall time: 1min 5s


The following shows the maximum likelihood estimators for $(\mu_{a}, \mu_{b}, \sigma^2_{a}, \sigma^2_{b})$, respectively:

In [12]:
print(opt_res2)
print('μ_a, μ_b, σ^2_a,  σ^2_b', μ_σ2[0], μ_σ2[1], μ_σ2[2],  μ_σ2[3])

      fun: 926329.6828881726
 hess_inv: array([[ 0.00178051, -0.00535074,  0.00018954, -0.00587788],
       [-0.00535074,  0.0651952 , -0.00394426,  0.05789564],
       [ 0.00018954, -0.00394426,  0.00122719, -0.00289736],
       [-0.00587788,  0.05789564, -0.00289736,  0.07986976]])
      jac: array([ 0.8203125,  0.2578125, -0.90625  , -0.3046875])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 642
      nit: 39
     njev: 105
   status: 2
  success: False
        x: array([  -5.3861355 ,   -2.99627238,    9.84261865, 2225.667973  ])
μ_a, μ_b, σ^2_a,  σ^2_b -5.386135496963321 -2.996272382154653 9.842618653237777 2225.667973001784


The following shows the standard errors for the estimators of $(\mu_{\alpha}, \mu_{\beta}, \sigma^2_{\alpha}, \sigma^2_{\beta})$, respectively:

In [13]:
print(se2)

[0.04219614 0.25533351 0.03503124 0.28261238]


The following shows the estimated tail index $\alpha$ using the estimators $(\mu_a, \sigma^2_a)$ and equation [(12)]() above.

In [14]:
α = fq.alpha(μ_σ2[0], μ_σ2[1])
print('The tail index α of the stationary firm size distribution is', α)

The tail index α of the stationary firm size distribution is -3.595224205277419


### 2.2.2 case ii: mle with most-recent observation for each firm in sample

Next, we keep the only most-recent observation for each firm to avoid time correlation.

In [15]:
df3 = df2.groupby('id', as_index=False).tail(1)
opt_res3, μ_σ3, se3 = fq.mle_mean_var_se(df3, x='logtasset_inv', y='fgrow_log')
df3

,year,id,tasset,logtasset,fgrow_log,logtasset_inv
8,2018,1,219295000.0,19.205928,-7.550921,0.052067
17,2018,2,162648000.0,18.907099,-32.561366,0.052890
26,2018,3,365725000.0,19.717392,7.731033,0.050717
35,2018,4,196456000.0,19.095949,-12.425937,0.052367
44,2018,5,346196000.0,19.662516,-4.628694,0.050858
...,...,...,...,...,...,...
197110,2017,39564,5133.0,8.543446,47.755202,0.117049
197111,2018,39566,971579.0,13.786678,87.515404,0.072534
197112,2018,39582,96026.0,11.472374,0.000000,0.087166
197114,2018,39592,99948.0,11.512405,0.000000,0.086863


In [16]:
df3.to_csv('data/usa_chop_log8_select.csv')

The following shows the maximum likelihood estimators for $(\mu_{a}, \mu_{b}, \sigma^2_{a}, \sigma^2_{b})$, respectively, in this new case:

In [17]:
print(opt_res3)
print('μ_a, μ_b, σ^2_a,  σ^2_b', μ_σ3[0], μ_σ3[1], μ_σ3[2],  μ_σ3[3])

      fun: 196697.305559264
 hess_inv: array([[ 9.96510536e-01, -2.80461067e-04,  5.90325806e-02,
         5.65993671e-04],
       [-2.80461067e-04,  9.99977478e-01,  4.75178384e-03,
         4.56400854e-05],
       [ 5.90325806e-02,  4.75178384e-03,  3.84015301e-03,
        -9.52241386e-03],
       [ 5.65993671e-04,  4.56400854e-05, -9.52241386e-03,
         9.99909300e-01]])
      jac: array([   977.71484375,     75.57421875, -17604.09960938,   -180.8671875 ])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 488
      nit: 1
     njev: 80
   status: 2
  success: False
        x: array([-5.67548509e-02, -4.42528401e-03,  1.10083427e+01,  1.00102153e+01])
μ_a, μ_b, σ^2_a,  σ^2_b -0.056754850924699 -0.004425284007126844 11.008342675829443 10.010215275771438


The following shows the standard errors for the estimators of $(\mu_{a}, \mu_{b}, \sigma^2_{a}, \sigma^2_{b})$, respectively:

In [18]:
print(se3)

[0.99825374 0.99998874 0.06196897 0.99995465]


The following shows the estimated tail index $\alpha$ using the estimators $(\mu_a, \sigma^2_a)$ and equation [(12)]() above.

In [19]:
α = fq.alpha(μ_σ3[0], μ_σ3[1])
print('The tail index α of the stationary firm size distribution is', α)

The tail index α of the stationary firm size distribution is -25.650263726936526


# 3 goodness-of-fit fest for MLE (TBD)

## 3.1 statistical inference based on MLE for data points with Gaussian uncertainties

A ML fit to data provides estimates for parameters and an error matrix. Inserting the estimates into the likelihood function yields a distribution that we want will model the data reasonably well. How well the data are modeled by that distribution is the goodness-of-fit.

In general, extra work is needed to obtain a quantitative measure of the goodness-of-fit, and there is no universally accepted mathematical definition which is valid in all cases.

According to Joel Heinrich's arguments (please see [here](https://www-cdf.fnal.gov/physics/statistics/recommendations/goodnessoffit.html)), if the data are represented by discrete points with Gaussian uncertainties, the chi-square test can be used to measure how well the fit matches the data.

Next, we will do the chi-square test for MLE we did by using the same sample.
- Definitions of [chi-square test](https://en.wikipedia.org/wiki/Chi-squared_test) and degrees of freedom can be find [here](http://onlinebooks.library.upenn.edu/webbin/book/lookupid?key=olbp12405).
- [This one](https://www.cambridge.org/core/books/statistics-for-nuclear-and-particle-physicists/9544B39F3244D9457BEC324CD34F1571) has an excellent discussion in $4.5$.
- [Numerical recipes]() provides some excellent advice on using chi-square for goodness-of-fit in $15.1$.